In [ ]:
import requests
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.types import BigInteger, Integer, Numeric, String, Boolean, DateTime
import os
from dotenv import load_dotenv
import time

# Завантаження змінних із .env
load_dotenv()

# Налаштування
API_URL = "https://openapi.keycrm.app/v1/order"
API_TOKEN = os.getenv("API_TOKEN")
PG_CONN = os.getenv("PG_CONN")
PER_PAGE = 50  # Максимальна кількість записів на сторінку

# Перевірка, чи завантажено змінні
if not API_TOKEN or not PG_CONN:
    raise ValueError("API_TOKEN або PG_CONN не знайдено в .env файлі")

# Ініціалізація підключення до PostgreSQL
engine = create_engine(PG_CONN)
headers = {"Authorization": f"Bearer {API_TOKEN}"}

def fetch_and_store_orders():
    page = 1
    while True:
        params = {"per_page": PER_PAGE, "page": page, "include": "buyer,products,payments,shipping,expenses,tags,status"}
        try:
            response = requests.get(API_URL, headers=headers, params=params)
            response.raise_for_status()  # Перевірка на помилки HTTP
        except requests.RequestException as e:
            print(f"Помилка API на сторінці {page}: {e}")
            break
        
        data = response.json()
        orders = data.get("data", [])
        if not orders:
            print(f"Дані закінчилися на сторінці {page}")
            break
        
        print(f"Обробка сторінки {page}, записів: {len(orders)}")
        
        # Ініціалізація списків для даних
        orders_list = []
        buyers_list = []
        statuses_list = []
        products_list = []
        properties_list = []
        payments_list = []
        shipping_list = []
        expenses_list = []
        tags_list = []
        
        for order in orders:
            try:
                # Перевірка, чи order не є None і має id
                if not order or not isinstance(order, dict) or "id" not in order:
                    print(f"Пропущено некоректне замовлення: {order}")
                    continue
                
                # Дані для orders
                orders_list.append({
                    "order_id": order.get("id"),
                    "parent_id": order.get("parent_id"),
                    "source_uuid": order.get("source_uuid"),
                    "source_id": order.get("source_id"),
                    "status_id": order.get("status_id"),
                    "status_group_id": order.get("status_group_id"),
                    "grand_total": order.get("grand_total"),
                    "promocode": order.get("promocode"),
                    "total_discount": order.get("total_discount"),
                    "expenses_sum": order.get("expenses_sum"),
                    "shipping_price": order.get("shipping_price"),
                    "wrap_price": order.get("wrap_price"),
                    "taxes": order.get("taxes"),
                    "manager_comment": order.get("manager_comment"),
                    "buyer_comment": order.get("buyer_comment"),
                    "gift_message": order.get("gift_message"),
                    "is_gift": order.get("is_gift"),
                    "payment_status": order.get("payment_status"),
                    "created_at": order.get("created_at"),
                    "ordered_at": order.get("ordered_at"),
                    "updated_at": order.get("updated_at"),
                    "closed_at": order.get("closed_at"),
                    "buyer_id": order.get("buyer", {}).get("id") if isinstance(order.get("buyer"), dict) else None
                })
                
                # Дані для buyers
                buyer = order.get("buyer")
                if buyer and isinstance(buyer, dict):
                    buyers_list.append({
                        "buyer_id": buyer.get("id"),
                        "full_name": buyer.get("full_name"),
                        "email": buyer.get("email"),
                        "phone": buyer.get("phone"),
                        "company_id": int(buyer.get("company_id")) if buyer.get("company_id") and str(buyer.get("company_id")).isdigit() else None,
                        "manager_id": int(buyer.get("manager_id")) if buyer.get("manager_id") and str(buyer.get("manager_id")).isdigit() else None
                    })
                
                # Дані для statuses
                status = order.get("status")
                if status and isinstance(status, dict):
                    statuses_list.append({
                        "status_id": status.get("id"),
                        "name": status.get("name"),
                        "alias": status.get("alias"),
                        "is_active": status.get("is_active"),
                        "group_id": status.get("group_id"),
                        "is_closing_order": status.get("is_closing_order"),
                        "is_reserved": status.get("is_reserved"),
                        "created_at": status.get("created_at"),
                        "updated_at": status.get("updated_at")
                    })
                
                # Дані для order_products та product_properties
                products = order.get("products", [])
                if isinstance(products, list):
                    for product in products:
                        if not product or not isinstance(product, dict):
                            print(f"Пропущено некоректний продукт у замовленні {order.get('id')}: {product}")
                            continue
                        product_id = product.get("offer", {}).get("product_id") if isinstance(product.get("offer"), dict) else product.get("id", 0)
                        products_list.append({
                            "order_id": order.get("id"),
                            "product_id": product_id,
                            "name": product.get("name"),
                            "sku": product.get("sku"),
                            "price": product.get("price"),
                            "price_sold": product.get("price_sold"),
                            "purchased_price": product.get("purchased_price"),
                            "discount_percent": product.get("discount_percent"),
                            "discount_amount": product.get("discount_amount"),
                            "total_discount": product.get("total_discount"),
                            "quantity": product.get("quantity"),
                            "unit_type": product.get("unit_type"),
                            "upsale": product.get("upsale"),
                            "comment": product.get("comment"),
                            "product_status_id": product.get("product_status_id"),
                            "shipment_type": product.get("shipment_type"),
                            "warehouse_id": product.get("warehouse", {}).get("id") if isinstance(product.get("warehouse"), dict) else None,
                            "warehouse_name": product.get("warehouse", {}).get("name") if isinstance(product.get("warehouse"), dict) else None
                        })
                        properties = product.get("properties", [])
                        if isinstance(properties, list):
                            for prop in properties:
                                if not prop or not isinstance(prop, dict):
                                    continue
                                properties_list.append({
                                    "order_id": order.get("id"),
                                    "product_id": product_id,
                                    "property_name": prop.get("name"),
                                    "property_value": prop.get("value")
                                })
                
                # Дані для payments
                payments = order.get("payments", [])
                if isinstance(payments, list):
                    for payment in payments:
                        if not payment or not isinstance(payment, dict):
                            continue
                        payments_list.append({
                            "payment_id": payment.get("id"),
                            "order_id": order.get("id"),
                            "destination_id": payment.get("destination_id"),
                            "payment_method_id": payment.get("payment_method_id"),
                            "amount": payment.get("amount"),
                            "actual_currency": payment.get("actual_currency"),
                            "transaction_uuid": payment.get("transaction_uuid"),
                            "description": payment.get("description"),
                            "status": payment.get("status"),
                            "payment_date": payment.get("payment_date"),
                            "created_at": payment.get("created_at"),
                            "updated_at": payment.get("updated_at")
                        })
                
                # Дані для shipping
                shipping = order.get("shipping")
                if shipping and isinstance(shipping, dict):
                    shipping_list.append({
                        "order_id": order.get("id"),
                        "delivery_service_id": shipping.get("delivery_service_id"),
                        "tracking_code": shipping.get("tracking_code"),
                        "shipping_status": shipping.get("shipping_status"),
                        "shipping_address_city": shipping.get("shipping_address_city"),
                        "shipping_address_country": shipping.get("shipping_address_country"),
                        "shipping_address_country_code": shipping.get("shipping_address_country_code"),
                        "shipping_address_region": shipping.get("shipping_address_region"),
                        "shipping_address_zip": shipping.get("shipping_address_zip"),
                        "shipping_secondary_line": shipping.get("shipping_secondary_line"),
                        "shipping_receive_point": shipping.get("shipping_receive_point"),
                        "recipient_full_name": shipping.get("recipient_full_name"),
                        "recipient_phone": shipping.get("recipient_phone"),
                        "shipping_date_actual": shipping.get("shipping_date_actual")
                    })
                
                # Дані для expenses
                expenses = order.get("expenses", [])
                if isinstance(expenses, list):
                    for expense in expenses:
                        if not expense or not isinstance(expense, dict):
                            continue
                        expenses_list.append({
                            "expense_id": expense.get("id"),
                            "order_id": order.get("id"),
                            "destination_id": expense.get("destination_id"),
                            "expense_type_id": expense.get("expense_type_id"),
                            "amount": expense.get("amount"),
                            "actual_currency": expense.get("actual_currency"),
                            "transaction_uuid": expense.get("transaction_uuid"),
                            "description": expense.get("description"),
                            "status": expense.get("status"),
                            "payment_date": expense.get("payment_date"),
                            "created_at": expense.get("created_at"),
                            "updated_at": expense.get("updated_at")
                        })
                
                # Дані для tags
                tags = order.get("tags", [])
                if isinstance(tags, list):
                    for tag in tags:
                        if not tag or not isinstance(tag, dict):
                            continue
                        tags_list.append({
                            "order_id": order.get("id"),
                            "tag_id": tag.get("id"),
                            "name": tag.get("name"),
                            "alias": tag.get("alias"),
                            "color": tag.get("color"),
                            "created_at": tag.get("created_at"),
                            "updated_at": tag.get("updated_at")
                        })
            
            except Exception as e:
                print(f"Помилка при обробці замовлення {order.get('id', 'невідоме')}: {e}")
                continue
        
        # Зберігання даних у PostgreSQL з ON CONFLICT
        try:
            with engine.connect() as conn:
                # Визначення типів даних для всіх таблиць
                buyers_dtype = {
                    "buyer_id": BigInteger,
                    "full_name": String(255),
                    "email": String(255),
                    "phone": String(50),
                    "company_id": BigInteger,
                    "manager_id": BigInteger
                }
                statuses_dtype = {
                    "status_id": BigInteger,
                    "name": String(255),
                    "alias": String(50),
                    "is_active": Boolean,
                    "group_id": BigInteger,
                    "is_closing_order": Boolean,
                    "is_reserved": Boolean,
                    "created_at": DateTime,
                    "updated_at": DateTime
                }
                orders_dtype = {
                    "order_id": BigInteger,
                    "parent_id": BigInteger,
                    "source_uuid": String(50),
                    "source_id": BigInteger,
                    "status_id": BigInteger,
                    "status_group_id": BigInteger,
                    "grand_total": Numeric(10, 2),
                    "promocode": String(50),
                    "total_discount": Numeric(10, 2),
                    "expenses_sum": Numeric(10, 2),
                    "shipping_price": Numeric(10, 2),
                    "wrap_price": Numeric(10, 2),
                    "taxes": Numeric(10, 2),
                    "manager_comment": String,
                    "buyer_comment": String,
                    "gift_message": String,
                    "is_gift": Boolean,
                    "payment_status": String(50),
                    "created_at": DateTime,
                    "ordered_at": DateTime,
                    "updated_at": DateTime,
                    "closed_at": DateTime,
                    "buyer_id": BigInteger
                }
                products_dtype = {
                    "order_id": BigInteger,
                    "product_id": BigInteger,
                    "name": String(255),
                    "sku": String(50),
                    "price": Numeric(10, 2),
                    "price_sold": Numeric(10, 2),
                    "purchased_price": Numeric(10, 2),
                    "discount_percent": Numeric(5, 2),
                    "discount_amount": Numeric(10, 2),
                    "total_discount": Numeric(10, 2),
                    "quantity": Integer,
                    "unit_type": String(50),
                    "upsale": Boolean,
                    "comment": String,
                    "product_status_id": BigInteger,
                    "shipment_type": String(50),
                    "warehouse_id": BigInteger,
                    "warehouse_name": String(255)
                }
                properties_dtype = {
                    "order_id": BigInteger,
                    "product_id": BigInteger,
                    "property_name": String(255),
                    "property_value": String  # Змінено на String без обмеження для TEXT
                }
                payments_dtype = {
                    "payment_id": BigInteger,
                    "order_id": BigInteger,
                    "destination_id": BigInteger,
                    "payment_method_id": BigInteger,
                    "amount": Numeric(10, 2),
                    "actual_currency": String(10),
                    "transaction_uuid": String(50),
                    "description": String,
                    "status": String(50),
                    "payment_date": DateTime,
                    "created_at": DateTime,
                    "updated_at": DateTime
                }
                shipping_dtype = {
                    "order_id": BigInteger,
                    "delivery_service_id": BigInteger,
                    "tracking_code": String(50),
                    "shipping_status": String(50),
                    "shipping_address_city": String(255),
                    "shipping_address_country": String(255),
                    "shipping_address_country_code": String(10),
                    "shipping_address_region": String(255),
                    "shipping_address_zip": String(50),
                    "shipping_secondary_line": String(255),
                    "shipping_receive_point": String(255),
                    "recipient_full_name": String(255),
                    "recipient_phone": String(50),
                    "shipping_date_actual": DateTime
                }
                expenses_dtype = {
                    "expense_id": BigInteger,
                    "order_id": BigInteger,
                    "destination_id": BigInteger,
                    "expense_type_id": BigInteger,
                    "amount": Numeric(10, 2),
                    "actual_currency": String(10),
                    "transaction_uuid": String(50),
                    "description": String,
                    "status": String(50),
                    "payment_date": DateTime,
                    "created_at": DateTime,
                    "updated_at": DateTime
                }
                tags_dtype = {
                    "order_id": BigInteger,
                    "tag_id": BigInteger,
                    "name": String(255),
                    "alias": String(50),
                    "color": String(10),
                    "created_at": DateTime,
                    "updated_at": DateTime
                }
                
                # Вставка даних із ON CONFLICT
                if buyers_list:
                    df_buyers = pd.DataFrame(buyers_list).drop_duplicates(subset=["buyer_id"])
                    df_buyers.to_sql("buyers_temp", conn, if_exists="replace", index=False, dtype=buyers_dtype)
                    conn.execute(text("""
                        INSERT INTO buyers
                        SELECT * FROM buyers_temp
                        ON CONFLICT (buyer_id) DO NOTHING
                    """))
                    conn.execute(text("DROP TABLE IF EXISTS buyers_temp"))
                
                if statuses_list:
                    df_statuses = pd.DataFrame(statuses_list).drop_duplicates(subset=["status_id"])
                    df_statuses.to_sql("statuses_temp", conn, if_exists="replace", index=False, dtype=statuses_dtype)
                    conn.execute(text("""
                        INSERT INTO statuses
                        SELECT * FROM statuses_temp
                        ON CONFLICT (status_id) DO NOTHING
                    """))
                    conn.execute(text("DROP TABLE IF EXISTS statuses_temp"))
                
                if orders_list:
                    df_orders = pd.DataFrame(orders_list).drop_duplicates(subset=["order_id"])
                    df_orders.to_sql("orders_temp", conn, if_exists="replace", index=False, dtype=orders_dtype)
                    conn.execute(text("""
                        INSERT INTO orders
                        SELECT * FROM orders_temp
                        ON CONFLICT (order_id) DO NOTHING
                    """))
                    conn.execute(text("DROP TABLE IF EXISTS orders_temp"))
                
                if products_list:
                    df_products = pd.DataFrame(products_list).drop_duplicates(subset=["order_id", "product_id"])
                    df_products.to_sql("order_products_temp", conn, if_exists="replace", index=False, dtype=products_dtype)
                    conn.execute(text("""
                        INSERT INTO order_products
                        SELECT * FROM order_products_temp
                        ON CONFLICT (order_id, product_id) DO NOTHING
                    """))
                    conn.execute(text("DROP TABLE IF EXISTS order_products_temp"))
                
                if properties_list:
                    df_properties = pd.DataFrame(properties_list).drop_duplicates(subset=["order_id", "product_id", "property_name"])
                    df_properties.to_sql("product_properties_temp", conn, if_exists="replace", index=False, dtype=properties_dtype)
                    conn.execute(text("""
                        INSERT INTO product_properties
                        SELECT * FROM product_properties_temp
                        ON CONFLICT (order_id, product_id, property_name) DO NOTHING
                    """))
                    conn.execute(text("DROP TABLE IF EXISTS product_properties_temp"))
                
                if payments_list:
                    df_payments = pd.DataFrame(payments_list).drop_duplicates(subset=["payment_id"])
                    df_payments.to_sql("payments_temp", conn, if_exists="replace", index=False, dtype=payments_dtype)
                    conn.execute(text("""
                        INSERT INTO payments
                        SELECT * FROM payments_temp
                        ON CONFLICT (payment_id) DO NOTHING
                    """))
                    conn.execute(text("DROP TABLE IF EXISTS payments_temp"))
                
                if shipping_list:
                    df_shipping = pd.DataFrame(shipping_list).drop_duplicates(subset=["order_id"])
                    df_shipping.to_sql("shipping_temp", conn, if_exists="replace", index=False, dtype=shipping_dtype)
                    conn.execute(text("""
                        INSERT INTO shipping
                        SELECT * FROM shipping_temp
                        ON CONFLICT (order_id) DO NOTHING
                    """))
                    conn.execute(text("DROP TABLE IF EXISTS shipping_temp"))
                
                if expenses_list:
                    df_expenses = pd.DataFrame(expenses_list).drop_duplicates(subset=["expense_id"])
                    df_expenses.to_sql("expenses_temp", conn, if_exists="replace", index=False, dtype=expenses_dtype)
                    conn.execute(text("""
                        INSERT INTO expenses
                        SELECT * FROM expenses_temp
                        ON CONFLICT (expense_id) DO NOTHING
                    """))
                    conn.execute(text("DROP TABLE IF EXISTS expenses_temp"))
                
                if tags_list:
                    df_tags = pd.DataFrame(tags_list).drop_duplicates(subset=["order_id", "tag_id"])
                    df_tags.to_sql("tags_temp", conn, if_exists="replace", index=False, dtype=tags_dtype)
                    conn.execute(text("""
                        INSERT INTO tags
                        SELECT * FROM tags_temp
                        ON CONFLICT (order_id, tag_id) DO NOTHING
                    """))
                    conn.execute(text("DROP TABLE IF EXISTS tags_temp"))
                
                conn.commit()
        except Exception as e:
            print(f"Помилка при збереженні даних на сторінці {page}: {e}")
            break
        
        # Перевірка наступної сторінки
        next_page_url = data.get("next_page_url")
        if not next_page_url:
            print(f"Вивантаження завершено на сторінці {page}")
            break
        page += 1
        time.sleep(1)  # Затримка для уникнення API-обмежень

if __name__ == "__main__":
    try:
        fetch_and_store_orders()
        print("Дані успішно вивантажено до PostgreSQL!")
    except Exception as e:
        print(f"Критична помилка: {e}")